In [10]:
!rm sa_tsp_seq.c


In [14]:
%%writefile sa_tsp_seq.c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <time.h>

#define N 100 // Number of cities

// Generate random distance matrix
void generate_distance_matrix(double dist[N][N]) {
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            if (i == j) dist[i][j] = 0;
            else dist[i][j] = rand() % 100 + 1;
        }
    }
}

// Compute total distance of a given tour
double tour_length(const int *tour, double dist[N][N]) {
    double total = 0.0;
    for (int i = 0; i < N; i++)
        total += dist[tour[i]][tour[(i + 1) % N]];
    return total;
}

// Swap two cities in the tour
void swap(int *tour, int i, int j) {
    if (i == j) return;
    int temp = tour[i];
    tour[i] = tour[j];
    tour[j] = temp;
}

// Shuffle tour randomly
void shuffle(int *tour) {
    for (int i = 0; i < N; i++) tour[i] = i;
    for (int i = N - 1; i > 0; --i) {
        int j = rand() % (i + 1);
        swap(tour, i, j);
    }
}

int main(void) {
    srand(12345);  // Fixed seed for reproducible results

    static double distance_matrix[N][N];
    generate_distance_matrix(distance_matrix);

    int current_tour[N], best_tour[N];
    shuffle(current_tour);
    double current_cost = tour_length(current_tour, distance_matrix);

    for (int i = 0; i < N; ++i) best_tour[i] = current_tour[i];
    double best_cost = current_cost;

    double T = 1000.0, T_min = 0.1, cooling = 0.999;
    const int max_iters = 200000;

    clock_t start = clock();

    for (int iter = 0; iter < max_iters && T > T_min; ++iter) {
        int i = rand() % N;
        int j = rand() % N;
        if (i == j) continue;

        swap(current_tour, i, j);
        double new_cost = tour_length(current_tour, distance_matrix);
        double delta = new_cost - current_cost;

        if (delta < 0 || exp(-delta / T) > (double)rand() / RAND_MAX) {
            current_cost = new_cost;
            if (current_cost < best_cost) {
                best_cost = current_cost;
                for (int k = 0; k < N; ++k)
                    best_tour[k] = current_tour[k];
            }
        } else {
            swap(current_tour, i, j);
        }

        T *= cooling;
    }

    clock_t end = clock();
    double elapsed = (double)(end - start) / CLOCKS_PER_SEC;

    printf("========================================\n");
    printf("   Simulated Annealing for TSP (Sequential)\n");
    printf("========================================\n");
    printf("Number of cities     : %d\n", N);
    printf("Best cost (distance) : %.2f\n", best_cost);
    printf("Execution time       : %.4f seconds\n", elapsed);
    printf("Best tour:\n");
    for (int i = 0; i < N; ++i) printf("%d ", best_tour[i]);
    printf("%d\n", best_tour[0]);
    printf("========================================\n");

    return 0;
}


Overwriting sa_tsp_seq.c


In [15]:
!gcc sa_tsp_seq.c -O2 -lm -o sa_seq
!./sa_seq


   Simulated Annealing for TSP (Sequential)
Number of cities     : 100
Best cost (distance) : 1158.00
Execution time       : 0.0032 seconds
Best tour:
40 26 96 33 87 56 90 34 38 69 75 89 32 79 1 21 84 2 91 13 30 0 92 53 97 82 47 95 74 7 3 60 25 14 35 29 76 15 61 36 17 16 73 18 52 42 67 50 80 55 4 54 27 77 49 8 48 72 31 99 51 45 23 93 24 85 20 64 88 83 58 6 59 70 63 11 94 86 5 12 37 22 28 65 44 57 81 39 71 78 98 19 9 43 10 66 62 68 46 41 40
